In [1]:
!pip install --no-deps ../input/ensemble-boxes-104/ensemble_boxes-1.0.4

Processing /kaggle/input/ensemble-boxes-104/ensemble_boxes-1.0.4
  Created wheel for ensemble-boxes: filename=ensemble_boxes-1.0.4-py3-none-any.whl size=14078 sha256=0888bcf60705e681fab67596305b4e205a622e5416e209923cb7712d916c3da5
  Stored in directory: /root/.cache/pip/wheels/66/11/27/bd684df8a75ad0e720fdc77d199e5876063925350905a64205
Successfully built ensemble-boxes


In [2]:
from tqdm.auto import tqdm
from fastai.vision.all import *
from ensemble_boxes import *

In [3]:
test_files = [
    '../input/siim-covid-19-effdet-infer-merge-best-folds/effdet3_sub_iou@0.6.csv',
    '../input/efnd5-sub-stage1-v1/effdet5_sub_iou_0.6.csv',
    '../input/new-yolov5-infer-thres-0-6-no-tta/yolo_v5_bench_no_tta.csv',
  #  '../input/siim-covid-19-yolov5-infer-no-tta-b4a9fa/yolo_v5_bench_no_tta.csv',
    '../input/yolo-mid-score/yolo_v5_bench_no_tta.csv'
 #   '../input/siim-covid-19-effdet-infer-merge-best-fold-f4eca0/effdet3_sub_iou@0.6.csv',
    #'../input/siim-effnetv2-l-cascadercnn-mmdetection-infer/sub_cascade_rcnn_public.csv'

]
print('# models', len(test_files))
print(test_files)

# models 4
['../input/siim-covid-19-effdet-infer-merge-best-folds/effdet3_sub_iou@0.6.csv', '../input/efnd5-sub-stage1-v1/effdet5_sub_iou_0.6.csv', '../input/new-yolov5-infer-thres-0-6-no-tta/yolo_v5_bench_no_tta.csv', '../input/yolo-mid-score/yolo_v5_bench_no_tta.csv']


In [4]:
pred_locations = {}

for model_index in range(len(test_files)):
    # load table
    with open(test_files[model_index], mode='r') as infile:
        # open reader
        reader = csv.reader(infile)
        # skip header
        next(reader, None)
        # loop through rows
        for rows in reader:
            # retrieve information
            filename = rows[0]
            # print(rows[0])
            parts = rows[1].split()
            # print(parts)
            assert len(parts) % 6 == 0
            locations = []
            for ind in range(len(parts) // 6):
                #label = int(parts[ind * 6])
                label = parts[ind * 6]
                label = 1 if label == 'opacity' else 0
                score = float(parts[ind * 6 + 1])
                location = int(float(parts[ind * 6 + 2])), int(float(parts[ind * 6 + 3])), \
                           int(float(parts[ind * 6 + 4])), int(float(parts[ind * 6 + 5]))
                if score > 0 and label < 14:
                    locations.append((label, score, location))
            if filename in pred_locations:
                pred_locations[filename].append(locations)
            else:
                pred_locations[filename] = [locations]

In [5]:
test_filenames = [*pred_locations]
test_filenames.sort()

In [6]:
def merge_boxes_from_models(intersection_thr):
    n_images = len(test_filenames)
    n_variants = len(test_files)

    result_boxes = []
    result_scores = []
    result_labels = []
    for ii in tqdm(range(n_images), dynamic_ncols=True):
        image_id = test_filenames[ii]
        pred_variants = pred_locations[image_id]
        assert len(pred_variants) == n_variants

        pred_boxes = []
        pred_scores = []
        pred_labels = []
        max_value = 10000
        for vi in range(n_variants):
            if len(pred_variants[vi]) > 0:
                cur_pred_labels, cur_pred_scores, cur_pred_boxes = list(zip(*pred_variants[vi]))
            else:
                cur_pred_labels = []
                cur_pred_scores = []
                cur_pred_boxes = []
            cur_pred_boxes = np.array(cur_pred_boxes, copy=False)
            cur_pred_scores = np.array(cur_pred_scores, copy=False)
            cur_pred_labels = np.array(cur_pred_labels, copy=False)

            # WBF expects the coordinates in 0-1 range.
            cur_pred_boxes = cur_pred_boxes / max_value

            pred_boxes.append(cur_pred_boxes)
            pred_scores.append(cur_pred_scores)
            pred_labels.append(cur_pred_labels)

        # Calculate WBF
        pred_boxes, pred_scores, pred_labels = weighted_boxes_fusion(
            pred_boxes,
            pred_scores,
            pred_labels,
            #weights=[0.25,0.3,0.25,0.2],
            conf_type = 'avg',


            iou_thr=intersection_thr,
            skip_box_thr=0.00002,
            allows_overflow=True
        )
        pred_boxes = np.round(pred_boxes * max_value).astype(int)

        assert len(pred_boxes) == len(pred_scores)
        assert len(pred_boxes) == len(pred_scores)
        assert len(pred_boxes) == len(pred_labels)
        result_boxes.append(pred_boxes)
        result_scores.append(pred_scores)
        result_labels.append(pred_labels)

    return result_boxes, result_scores, result_labels

In [7]:
result_boxes, result_scores, result_labels = merge_boxes_from_models(0.625)

  0%|          | 0/1263 [00:00<?, ?it/s]

In [8]:
def format_prediction_string(boxes, scores, labels):
    pred_strings = []
    if len(boxes) > 0 :
        for j in zip(labels, scores, boxes):
            pred_strings.append("opacity {0:.4f} {1} {2} {3} {4}".format(j[1], j[2][0], j[2][1], j[2][2], j[2][3]))
    else:
        pred_strings.append("none 1 0 0 1 1")
    return " ".join(pred_strings)


def write_submission(test_images_ids, result_boxes, result_scores, result_labels, score_thr):
    results = []
    for i, image in enumerate(test_images_ids):
        image_id   = test_images_ids[i]
        cur_boxes  = np.array(result_boxes[i])
        cur_scores = np.array(result_scores[i])
        cur_labels = np.array(result_labels[i])
        score_filter = cur_scores >= score_thr
        cur_boxes  = cur_boxes[score_filter]
        cur_scores = cur_scores[score_filter]
        cur_labels = cur_labels[score_filter]
        result = {'image_id': image_id, 'PredictionString': format_prediction_string(cur_boxes, cur_scores, cur_labels)}
        results.append(result)
    return pd.DataFrame(results, columns=['image_id', 'PredictionString'])

In [9]:
sub = write_submission(test_filenames, result_boxes, result_scores, result_labels, 0)
sub.head()

,image_id,PredictionString
0,0026720152f5_image,opacity 0.3150 454 1125 1125 2236 opacity 0.2620 443 1655 1113 2267 opacity 0.2506 1851 1344 2422 2281 opacity 0.2126 2126 1874 2421 2277 opacity 0.1587 1929 1802 2418 2316 opacity 0.1371 1743 1657 2388 2343 opacity 0.1129 2069 1556 2443 2292 opacity 0.0767 423 1407 880 2225 opacity 0.0670 1669 902 2457 2319 opacity 0.0598 649 1392 1164 2176 opacity 0.0588 536 1127 977 2016 opacity 0.0562 1771 1848 2224 2303 opacity 0.0517 474 1784 891 2213 opacity 0.0488 559 1138 904 1502 opacity 0.0481 1669 1436 2223 2280 opacity 0.0441 2190 1924 2377 2159 opacity 0.0437 388 1658 844 2292 opacity 0.0429 ...
1,00fc8fc35dc1_image,opacity 0.5260 2322 869 3281 2430 opacity 0.4937 613 631 1598 2325 opacity 0.1417 697 723 1490 1725 opacity 0.1370 607 1360 1478 2337 opacity 0.0981 594 861 1243 2165 opacity 0.0871 2686 1054 3290 2357 opacity 0.0589 2433 1300 3163 2332 opacity 0.0575 2045 285 3359 2716 opacity 0.0418 685 794 1168 1567 opacity 0.0345 2257 1403 3356 2655 opacity 0.0321 787 997 1510 2114 opacity 0.0290 400 197 1847 2748 opacity 0.0255 971 961 1668 2306 opacity 0.0251 512 1284 1622 2651 opacity 0.0242 2218 617 3105 2060 opacity 0.0238 744 1638 1339 2218 opacity 0.0219 669 946 1218 1697 opacity 0.0214 2602 872...
2,01c3512eebc3_image,opacity 0.2304 402 1311 1074 1952 opacity 0.2021 1771 1485 2345 1992 opacity 0.1620 1953 1604 2354 1974 opacity 0.1009 408 733 1101 1910 opacity 0.0945 1703 813 2322 1909 opacity 0.0920 387 1472 884 1945 opacity 0.0901 1894 1123 2368 1955 opacity 0.0851 360 1024 863 1908 opacity 0.0837 1982 1666 2298 1926 opacity 0.0765 1680 1413 2187 1942 opacity 0.0761 2065 1608 2395 2028 opacity 0.0624 666 1176 1143 1831 opacity 0.0568 330 1592 659 1992 opacity 0.0548 307 1432 703 1998 opacity 0.0529 336 1279 804 1982 opacity 0.0487 1659 1120 2134 1899 opacity 0.0479 573 1364 1142 1904 opacity 0.0478 20...
3,01f948f8e544_image,opacity 0.2525 574 1078 1153 1879 opacity 0.2111 1887 912 2411 1828 opacity 0.0921 708 1435 1188 1896 opacity 0.0772 756 1213 1250 1866 opacity 0.0720 497 1011 962 1886 opacity 0.0701 1782 835 2315 1570 opacity 0.0654 609 1024 1016 1642 opacity 0.0622 588 978 831 1360 opacity 0.0608 2064 916 2423 1617 opacity 0.0587 1711 1051 2283 1917 opacity 0.0570 588 1034 907 1482 opacity 0.0569 1680 591 2463 1907 opacity 0.0554 1964 962 2366 1533 opacity 0.0552 548 642 1293 1933 opacity 0.0496 1843 1312 2423 2005 opacity 0.0477 2050 1034 2484 1966 opacity 0.0449 548 1023 897 1781 opacity 0.0438 1746 1...
4,022146012034_image,opacity 0.3527 386 1321 1065 1903 opacity 0.2690 1903 1448 2462 2093 opacity 0.0792 1741 1350 2309 2064 opacity 0.0791 602 1299 1155 1874 opacity 0.0785 1806 944 2501 2113 opacity 0.0783 364 817 1088 1912 opacity 0.0709 1984 1596 2342 2046 opacity 0.0659 295 1281 843 1927 opacity 0.0622 536 1371 964 1806 opacity 0.0570 1909 1676 2383 2119 opacity 0.0504 2025 1593 2545 2191 opacity 0.0491 2059 1312 2558 2083 opacity 0.0410 1783 1579 2317 2103 opacity 0.0394 1976 1124 2480 1992 opacity 0.0374 229 1211 1202 2093 opacity 0.0323 2040 1411 2426 1922 opacity 0.0298 1723 1378 2611 2250 opacity 0.0...


In [10]:
sub.to_csv('yolov5_effdet5_iou0.6.csv', index=False)